In [ ]:
 # Import the libraries.
import requests
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import csv
import matplotlib.pyplot as plt
import time
import random
from google.colab import drive
import re

In [ ]:
# collegamento a google drive
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/ristoranti_tripadvisor.csv"
df= pd.read_csv(file_path)
df

Mounted at /content/drive


,name,url,rating,price,contact,via,CAP,comune
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+393932505982,Corso Milano 17,24020.0,Selvino
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+39035763104,Via Talpino 79,24020.0,Selvino
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+39035763012,Via Monte Alben 13,24020.0,Selvino
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,fascia media,+39035779167,Via Cantul 38,24020.0,Aviatico
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,cibo economico,+39035764202,Corso Milano 91,24020.0,Selvino
...,...,...,...,...,...,...,...,...
879,Ristorante Al Caminone,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+393486626501,Via S. Pellico 3 Hotel Milano Alpen Resort,24020.0,Castione della Presolana
880,Bar Del Parco,https://www.tripadvisor.it/Restaurant_Review-g...,1.0,NaN,NaN,Parco Degli Alpini,NaN,Castione della Presolana
881,Gastronomia Sapori di montagna,https://www.tripadvisor.it/Restaurant_Review-g...,5.0,fascia media,+393496868491,Via Andrea Fantoni 75,24020.0,Castione della Presolana
882,Baita Valle Azzurra,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cibo economico,+39034628193,Via Valzurio 9 Dopo la chiesa sulla sinistra,24020.0,Oltressenda Alta


In [ ]:
df = df.rename(columns={'comune': 'city', 'via': 'locations','name': 'titles', 'rating':'trip_rating', 'price': 'trip_price' })
df['data_source']='tripadvisor'
df

,titles,url,trip_rating,trip_price,contact,locations,CAP,city,data_source
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+393932505982,Corso Milano 17,24020.0,Selvino,tripadvisor
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+39035763104,Via Talpino 79,24020.0,Selvino,tripadvisor
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+39035763012,Via Monte Alben 13,24020.0,Selvino,tripadvisor
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,fascia media,+39035779167,Via Cantul 38,24020.0,Aviatico,tripadvisor
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,cibo economico,+39035764202,Corso Milano 91,24020.0,Selvino,tripadvisor
...,...,...,...,...,...,...,...,...,...
879,Ristorante Al Caminone,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+393486626501,Via S. Pellico 3 Hotel Milano Alpen Resort,24020.0,Castione della Presolana,tripadvisor
880,Bar Del Parco,https://www.tripadvisor.it/Restaurant_Review-g...,1.0,NaN,NaN,Parco Degli Alpini,NaN,Castione della Presolana,tripadvisor
881,Gastronomia Sapori di montagna,https://www.tripadvisor.it/Restaurant_Review-g...,5.0,fascia media,+393496868491,Via Andrea Fantoni 75,24020.0,Castione della Presolana,tripadvisor
882,Baita Valle Azzurra,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cibo economico,+39034628193,Via Valzurio 9 Dopo la chiesa sulla sinistra,24020.0,Oltressenda Alta,tripadvisor


In [ ]:
# open file coordinate geografiche
file_path_3 = '/content/drive/MyDrive/italy_geo.xlsx'

df_geo = pd.read_excel(file_path_3, sheet_name="italy_geo")

In [ ]:
# Rimuovi gli spazi extra dalle colonne "city" nel primo dataset
df['city'] = df['city'].str.strip()

# Rimuovi gli spazi extra dalle colonne "comune" nel secondo dataset
df_geo['comune'] = df_geo['comune'].str.strip()

In [ ]:
df_geo

,istat,comune,lng,lat
0,1001,Agliè,7.768600,45.363433
1,1002,Airasca,7.484431,44.916886
2,1003,Ala di Stura,7.304344,45.315110
3,1004,Albiano d'Ivrea,7.949145,45.433893
4,1005,Alice Superiore,7.777019,45.460094
...,...,...,...,...
7974,111104,Villasalto,9.388774,39.493758
7975,111105,Villasimius,9.520365,39.142384
7976,111106,Villasor,8.940650,39.380689
7977,111107,Villaspeciosa,8.926934,39.312805


In [ ]:
# Unione dei dataset basata su "city" e "Comune" mantenendo solo le righe che sono presenti in "City"
merged_df = pd.merge(df, df_geo, left_on='city', right_on='comune', how='left')
merged_df

,titles,url,trip_rating,trip_price,contact,locations,CAP,city,data_source,istat,comune,lng,lat
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+393932505982,Corso Milano 17,24020.0,Selvino,tripadvisor,16197,Selvino,9.750127,45.780748
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+39035763104,Via Talpino 79,24020.0,Selvino,tripadvisor,16197,Selvino,9.750127,45.780748
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+39035763012,Via Monte Alben 13,24020.0,Selvino,tripadvisor,16197,Selvino,9.750127,45.780748
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,fascia media,+39035779167,Via Cantul 38,24020.0,Aviatico,tripadvisor,16015,Aviatico,9.768922,45.792805
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,cibo economico,+39035764202,Corso Milano 91,24020.0,Selvino,tripadvisor,16197,Selvino,9.750127,45.780748
...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,Ristorante Al Caminone,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+393486626501,Via S. Pellico 3 Hotel Milano Alpen Resort,24020.0,Castione della Presolana,tripadvisor,16064,Castione della Presolana,10.036409,45.907746
880,Bar Del Parco,https://www.tripadvisor.it/Restaurant_Review-g...,1.0,NaN,NaN,Parco Degli Alpini,NaN,Castione della Presolana,tripadvisor,16064,Castione della Presolana,10.036409,45.907746
881,Gastronomia Sapori di montagna,https://www.tripadvisor.it/Restaurant_Review-g...,5.0,fascia media,+393496868491,Via Andrea Fantoni 75,24020.0,Castione della Presolana,tripadvisor,16064,Castione della Presolana,10.036409,45.907746
882,Baita Valle Azzurra,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cibo economico,+39034628193,Via Valzurio 9 Dopo la chiesa sulla sinistra,24020.0,Oltressenda Alta,tripadvisor,16147,Oltressenda Alta,9.944216,45.914482


In [ ]:
#elimino colonne che non mi servono
merged_df = merged_df.drop(['comune', 'istat'], axis=1)


In [ ]:
print(merged_df.isna().sum())

# Rimuovi i record contenenti valori NaN
merged_df = merged_df.dropna()
merged_df

titles         0
url            0
trip_rating    0
trip_price     0
contact        0
locations      0
CAP            0
city           0
data_source    0
lng            0
lat            0
dtype: int64


,titles,url,trip_rating,trip_price,contact,locations,CAP,city,data_source,lng,lat
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+393932505982,Corso Milano 17,24020.0,Selvino,tripadvisor,9.750127,45.780748
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+39035763104,Via Talpino 79,24020.0,Selvino,tripadvisor,9.750127,45.780748
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+39035763012,Via Monte Alben 13,24020.0,Selvino,tripadvisor,9.750127,45.780748
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,fascia media,+39035779167,Via Cantul 38,24020.0,Aviatico,tripadvisor,9.768922,45.792805
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,4.0,cibo economico,+39035764202,Corso Milano 91,24020.0,Selvino,tripadvisor,9.750127,45.780748
...,...,...,...,...,...,...,...,...,...,...,...
876,Il Casolare,https://www.tripadvisor.it/Restaurant_Review-g...,3.5,fascia media,+39034636510,Via Rucola 8,24020.0,Castione della Presolana,tripadvisor,10.036409,45.907746
879,Ristorante Al Caminone,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cucina raffinata,+393486626501,Via S. Pellico 3 Hotel Milano Alpen Resort,24020.0,Castione della Presolana,tripadvisor,10.036409,45.907746
881,Gastronomia Sapori di montagna,https://www.tripadvisor.it/Restaurant_Review-g...,5.0,fascia media,+393496868491,Via Andrea Fantoni 75,24020.0,Castione della Presolana,tripadvisor,10.036409,45.907746
882,Baita Valle Azzurra,https://www.tripadvisor.it/Restaurant_Review-g...,4.5,cibo economico,+39034628193,Via Valzurio 9 Dopo la chiesa sulla sinistra,24020.0,Oltressenda Alta,tripadvisor,9.944216,45.914482


In [ ]:
# open file comprensori
file_path_4 = '/content/drive/MyDrive/comprensori_modificato.csv'

df_coordinate = pd.read_csv(file_path_4)

In [ ]:
#estraggo i comuni e li trasformo in un dizionario
comuni = df_coordinate['comune'].drop_duplicates()
city_names = comuni.to_dict()

In [ ]:
# esempio di calcolo della distanza per attribuzione dei punti
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

def get_coordinates(city):
    city = city.lower()  # Converti il nome della città in lettere minuscole
    row = df_geo.loc[df_geo['comune'].str.lower() == city]  # Confronta i nomi delle città in lettere minuscole
    if not row.empty:
        return {
            'latitude': row['lat'].values[0],
            'longitude': row['lng'].values[0]
        }
    else:
        return None

# Estrarre i punti di riferimento dalle città nel dizionario
reference_points = []
for key, city in city_names.items():
    city = city.lower()  # Converti il nome della città in lettere minuscole
    coordinates = get_coordinates(city)
    if coordinates is not None:
        reference_point = {
            'city': city,  # Aggiungi il nome della città al dizionario
            'latitude': coordinates['latitude'],
            'longitude': coordinates['longitude']
        }
        reference_points.append(reference_point)

        #reference_points.append(coordinates)

In [ ]:
reference_points

[{'city': 'ponte di legno', 'latitude': 46.25647207, 'longitude': 10.5114586},
 {'city': 'gromo', 'latitude': 45.97038555, 'longitude': 9.92990165},
 {'city': 'selvino', 'latitude': 45.78074838, 'longitude': 9.75012725},
 {'city': 'gerola alta', 'latitude': 46.06002756, 'longitude': 9.55022548},
 {'city': 'pisogne', 'latitude': 45.80480282, 'longitude': 10.10889801},
 {'city': 'piazzatorre', 'latitude': 45.99001243, 'longitude': 9.68557205},
 {'city': 'borno', 'latitude': 45.94679424, 'longitude': 10.20580785},
 {'city': 'chiesa in valmalenco',
  'latitude': 46.26376909,
  'longitude': 9.84832637},
 {'city': 'schilpario', 'latitude': 46.00956474, 'longitude': 10.15451561},
 {'city': 'aprica', 'latitude': 46.15334799, 'longitude': 10.15206343},
 {'city': 'livigno', 'latitude': 46.53867334, 'longitude': 10.13481042},
 {'city': 'artogne', 'latitude': 45.84832151, 'longitude': 10.16452435},
 {'city': 'bagolino', 'latitude': 45.82122915, 'longitude': 10.46893818},
 {'city': 'bormio', 'latit

In [ ]:
# Funzione per calcolare la distanza geografica tra due punti
def calculate_distance(row, reference_points):
    point = (row['lat'], row['lng'])  # Latitudine e longitudine del record
    distances = []

    for reference_point in reference_points:
        reference_coordinates = (reference_point['latitude'], reference_point['longitude'])  # Latitudine e longitudine del punto di riferimento
        distance = geodesic(point, reference_coordinates).kilometers  # Calcolo della distanza in chilometri
        distances.append(distance)

    min_distance_index = distances.index(min(distances))  # Indice della distanza minima
    nearest_reference = reference_points[min_distance_index]['city']
  # Punto di riferimento più vicino (nome città)

    return nearest_reference

# Aggiunta della colonna "nearest_reference" al DataFrame con il punto di riferimento più vicino
merged_df['nearest_reference'] = merged_df.apply(lambda row: calculate_distance(row, reference_points), axis=1)

In [ ]:
file_path_merged = '/content/drive/MyDrive/advisor_merged.csv'
merged_df.to_csv(file_path_merged, index=False)